
https://docs.cohere.com/v2/docs/build-things-with-cohere

# 1. 初期セットアップ


In [ ]:
import logging
import os
import json
from dotenv import load_dotenv
import cohere

load_dotenv()
co = cohere.ClientV2(api_key=os.environ['COHERE_API_KEY'])

# 2. テキスト生成

`chat` にメッセージを渡す。


In [ ]:
# Add the user message
message = """今日から私は新しいスタートアップ企業 Co1t に入社しました。
同僚に向けて、手短な自己紹介メッセージを書く手伝いをしてください。"""

# Generate the response
response = co.chat(
    model="command-a-03-2025",
    messages=[{"role": "user", "content": message}],
)
#    messages=[cohere.UserMessage(content=message)])

print(response.message.content[0].text)


## プロンプトエンジニアリング

上の実行例では、応答例を与えておらずゼロショットプロンプト (zero-shot prompting)。

応答の例示を与える few-shot prompting をしてみると、以下のような挙動になる。
(例示が英語なので、英訳も行われている)

In [ ]:
# Add the user message
user_input = (
    "どうして私はサーバにアクセスできないのですか？これは、権限設定の問題ですか？"
)

# Create a prompt containing example outputs
message = f"""以下のユーザ依頼に倣って、チケットタイトルを記述してください。:

User request: プロジェクトファイルを保管するストレージ場所はどこですか？
Ticket title: Project File Storage Location

User request: Eメールが送れません。何が問題ですか？
Ticket title: Email Sending Issues

User request: オフィスプリンタへの接続はどうすれば良い？
Ticket title: Printer Connection Setup

User request: {user_input}
Ticket title:"""

# Generate the response
response = co.chat(
    model="command-a-03-2025",
    messages=[{"role": "user", "content": message}],
)

print(response.message.content[0].text)

# [2026-01-30] Server Access Issue: Potential Permissions Problem

## パラメータによる出力制御

`temperature` を `0` にしてみると、

In [ ]:
# Add the user message
message = """私は、日本の自然風土とそれが現代世界にどのような影響を与えたかを学ぶのが好きです。
30字以内で自己紹介をお願いします。"""

# Generate the response multiple times by specifying a low temperature value
for idx in range(3):
    response = co.chat(
        model="command-a-03-2025",
        messages=[{"role": "user", "content": message}],
        temperature=0,
    )

    print(f"{idx+1}: {response.message.content[0].text}\n")


In [ ]:
# Generate the response multiple times by specifying a high temperature value
for idx in range(3):
    response = co.chat(
        model="command-a-03-2025",
        messages=[{"role": "user", "content": message}],
        temperature=1,
    )

    print(f"{idx+1}: {response.message.content[0].text}\n")

## 出力の構造化

`response_format`

In [ ]:
# Add the user message
user_input = (
    "どうしてサーバにアクセスできないの？アクセス権限の問題？"
)
message = f"""Create an IT ticket for the following user request. Generate a JSON object.
{user_input}"""

# Generate the response multiple times by adding the JSON schema
response = co.chat(
    model="command-a-03-2025",
    messages=[{"role": "user", "content": message}],
    response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "required": ["title", "category", "status"],
            "properties": {
                "title": {"type": "string"},
                "category": {
                    "type": "string",
                    "enum": ["access", "software"],
                },
                "status": {
                    "type": "string",
                    "enum": ["open", "closed"],
                },
            },
        },
    },
)

json_object = json.loads(response.message.content[0].text)

print(json.dumps(json_object, indent=2, ensure_ascii=False))


## 応答のストリーム化

`chat_stream` を使う。

In [ ]:
# Add the user message
message = """今日から私は新しいスタートアップ企業 Co1t に入社しました。
同僚に向けて、手短な自己紹介メッセージを書く手伝いをしてください。"""

# Generate the response by streaming it
response = co.chat_stream(
    model="command-a-03-2025",
    messages=[{"role": "user", "content": message}],
)

for event in response:
    if event:
        if event.type == "content-delta":
            print(event.delta.message.content.text, end="")